## Proyecto: **Sistema Seguro de Gestión de Cuentas y Tareas**

**Framework principal:** FastAPI

**GUI para configuración:** Streamlit

**Autenticación:** JWT

**Base de datos:** PostgreSQL 

**Logs:** Loguru

**Documentación:** Swagger (FastAPI)

**Protección por roles:** `admin` y `user`



---


## Estructura general por capas


```
app/
├── main.py                  # Entry point FastAPI
├── streamlit_app.py             # GUI para configurar la clave
├── requirements.txt
├── auth/
│   ├── auth_handler.py      # lógica JWT
│   ├── auth_service.py      # hash/verify
│   ├── dependencies.py      # middleware para FastAPI
├── core/
│   ├── database.py          # conexión a PostgreSQL       
│   ├── logger.py            # Logger con Loguru
├── models/
│   └── user_model.py              # modelo ORM
├── routes/
│   └── user_routes.py       # rutas
├── schemas/
│   └── user_schemas.py       # rutas

```

---


Veamos capa por capa:


### **1. `main.py`**

Es el **punto de entrada** de la aplicación. Aquí se:

* Inicializa la app FastAPI.
* Se configuran las rutas (importadas desde `routes/`).
* Se agregan middlewares globales (CORS, logging, etc.).
* Se levanta el servidor.

---

### **2. `auth/` – Capa de autenticación**

```
├── auth/
│   ├── auth_handler.py      # lógica JWT
│   ├── auth_service.py      # hash/verify
│   ├── dependencies.py      # middleware para FastAPI
```

Contiene **toda la lógica relacionada con seguridad y autenticación**, separada del resto del código.

* **`auth_handler.py`**
  Maneja la generación y verificación de **tokens JWT (JSON Web Tokens)**.
  Aquí se definen funciones como `create_access_token()` y `decode_token()`.

* **`auth_service.py`**
  Encargado de **hashear contraseñas**, verificarlas y posiblemente manejar el login.

* **`dependencies.py`**
  Define **dependencias de seguridad** para FastAPI, como `get_current_user()` o validadores de token que se inyectan en las rutas.

---

###  **3. `core/` – Capa de configuración y servicios básicos**

```
├── core/
│   ├── database.py          # conexión a PostgreSQL       
│   ├── logger.py            # Logger con Loguru
```

* **`database.py`**
  Aquí se establece la **conexión a la base de datos** (por ejemplo, PostgreSQL).
  Define el motor SQLAlchemy, la sesión y la base `Base` del ORM.

* **`logger.py`**
  Inicializa la configuraciòn del logging.

---

### **4. `models/` – Capa de datos (ORM)**

```
├── models/
│   └── user_model.py              # modelo ORM
```

* **`user_model.py`**
  Define los **modelos ORM** que representan las tablas de la base de datos.
  Por ejemplo, la tabla `users` con sus columnas y tipos de datos.

### **5. `routes/` – Capa de controladores o endpoints**

```
├── routes/
│   └── user_routes.py       # rutas
```

* **`user_routes.py`**
  Define las **rutas HTTP (GET, POST, PUT, DELETE)** que los clientes pueden consumir.
  Aquí se importa la lógica de `auth`, `schemas`, `models` y `database`.

---

### **6. `schemas/` – Capa de validación y transferencia de datos (Pydantic)**

```
├── schemas/
│   └── user_schemas.py       # rutas
```

* **`user_schemas.py`**
  Define los **esquemas Pydantic**, que se usan para validar la entrada y salida de datos en las rutas (no son tablas, sino estructuras de datos para la API).

---


### **Resumen de las capas:**

| Capa         | Propósito                           | Ejemplo de archivo |
| ------------ | ----------------------------------- | ------------------ |
| **main.py**  | Entrada principal de la app         | `main.py`          |
| **auth/**    | Manejo de autenticación y seguridad | `auth_handler.py`  |
| **core/**    | Configuración base del sistema      | `database.py`      |
| **models/**  | Modelos ORM (tablas)                | `user.py`          |
| **schemas/** | Validación de datos (Pydantic)      | `user_schemas.py`  |
| **routes/**  | Endpoints y lógica de negocio       | `user_routes.py`   |

---


Requisitos (requirements.txt)

In [ ]:
fastapi
uvicorn[standard]
sqlalchemy
pydantic
passlib[bcrypt]
python-jose[cryptography]
python-dotenv
loguru
streamlit
psycopg2-binary
requests



## Paso 1: **`core/` – Capa de configuración y servicios básicos**

* **`database.py`**
  Aquí se establece la **conexión a la base de datos** (por ejemplo, PostgreSQL).
  Define el motor SQLAlchemy, la sesión y la base `Base` del ORM.

---


In [ ]:
from sqlalchemy import create_engine  # Permite crear el motor de conexión a la base de datos
from sqlalchemy.ext.declarative import declarative_base  # Permite definir clases ORM que se traducen a tablas
from sqlalchemy.orm import sessionmaker  # Permite crear sesiones para interactuar con la base de datos
from sqlalchemy.exc import SQLAlchemyError  # Para capturar errores específicos de SQLAlchemy

# Configuración directa de PostgreSQL
DB_USER = "postgres"  # Usuario de la base de datos
DB_PASS = "postgres"  # Contraseña del usuario
DB_HOST = "localhost"  # Dirección del host de la base de datos
DB_PORT = "5432"  # Puerto donde escucha PostgreSQL
DB_NAME = "autenticacion"  # Nombre de la base de datos

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(DATABASE_URL)

    SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

    Base = declarative_base()

except SQLAlchemyError as e:
    raise Exception(f"Error al conectar con la base de datos: {str(e)}")

def get_db():
    db = SessionLocal()  # Se crea una nueva sesión a partir de SessionLocal
    try:
        yield db  # Se "entrega" la sesión al bloque que la necesite (por ejemplo, un endpoint)
    except SQLAlchemyError as e:
        db.rollback()  # Si ocurre un error, se hace rollback de cualquier cambio no confirmado
        raise Exception(f"Error en la operación de base de datos: {str(e)}")  # Se lanza una excepción personalizada
    finally:
        db.close()  # Siempre se cierra la sesión, haya error o no, para liberar recursos


* **`logger.py`**
  Inicializa la configuraciòn del logging.

---

In [ ]:
import logging
import os
from logging.handlers import TimedRotatingFileHandler

def init_logger():
    os.makedirs('logs', exist_ok=True)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    handler = TimedRotatingFileHandler('logs/app.log', when='midnight', backupCount=7, encoding='utf-8')
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    if not logger.handlers:
        logger.addHandler(handler)


# **2. `auth/` – Capa de autenticación**

Contiene **toda la lógica relacionada con seguridad y autenticación**, separada del resto del código.



## Paso 2: **`app/auth/auth_service.py`**

**`auth_service.py`**
  Encargado de **hashear contraseñas**, verificarlas y posiblemente manejar el login.


In [ ]:
from passlib.context import CryptContext

# Contexto de encriptación y base de datos de usuarios
HASH_SCHEME = "argon2"

pwd_context = CryptContext(schemes=[HASH_SCHEME], deprecated="auto")

def hashear_password(password: str) -> str:
    return pwd_context.hash(password)

def verificar_password(plain_password: str, hashed_password: str) -> bool:
    return pwd_context.verify(plain_password, hashed_password)


---

## Paso 3: **`app/auth/auth_handler.py`**

**`auth_handler.py`**
  Maneja la generación y verificación de **tokens JWT (JSON Web Tokens)**.
  Aquí se definen funciones como `create_access_token()` y `decode_token()`.




In [ ]:
from datetime import datetime, timedelta, timezone
from jose import jwt, JWTError
import os
import secrets
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configuración JWT desde variables de entorno
SECRET_KEY = os.getenv("SECRET_KEY")
if not SECRET_KEY:
    # Generar SECRET_KEY automáticamente con secrets si no existe
    SECRET_KEY = secrets.token_urlsafe(32)
    print(f"⚠️  ADVERTENCIA: SECRET_KEY no encontrada en .env")
    print(f"🔑 Usando clave temporal generada: {SECRET_KEY}")
    print(f"💡 Para producción, agrega esta clave a tu archivo .env")

ALGORITHM = os.getenv("ALGORITHM", "HS256")
ACCESS_TOKEN_EXPIRE_MINUTES = int(os.getenv("ACCESS_TOKEN_EXPIRE_MINUTES", "30"))

def crear_token(data: dict, expiracion: int = ACCESS_TOKEN_EXPIRE_MINUTES):
    to_encode = data.copy()
    
    expire = datetime.now(timezone.utc) + timedelta(minutes=expiracion)
    
    to_encode.update({"exp": expire})
    
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def verificar_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except JWTError:
        return None


---

## Paso 4: **`app/auth/dependencies.py`**

**`dependencies.py`**
    Define **dependencias de seguridad** para FastAPI, como `get_current_user()` o validadores de token que se inyectan en las rutas.

---


In [ ]:
# app/auth/dependencies.py

from fastapi import Depends, HTTPException, status
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from sqlalchemy.orm import Session
from core.database import get_db
from models.user_model import User
from auth.auth_handler import verificar_token

# Esquema de seguridad HTTP Bearer (para tokens JWT)
security = HTTPBearer()

async def get_current_user(
    credentials: HTTPAuthorizationCredentials = Depends(security),
    db: Session = Depends(get_db)
) -> User:
    """
    Dependency para obtener el usuario actual autenticado desde el token JWT.
    
    Args:
        credentials: Credenciales HTTP Bearer (token JWT)
        db: Sesión de base de datos
        
    Returns:
        User: Usuario autenticado
        
    Raises:
        HTTPException: Si el token es inválido o el usuario no existe
    """
    # Extraer el token del header Authorization
    token = credentials.credentials
    
    # Verificar y decodificar el token
    payload = verificar_token(token)
    
    if not payload:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token inválido o expirado",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    # Obtener el username del payload
    username: str = payload.get("sub")
    if not username:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token inválido: falta información del usuario",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    # Buscar el usuario en la base de datos
    user = db.query(User).filter(User.username == username).first()
    
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Usuario no encontrado",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    return user


---

## Paso 5. **`models/` – Capa de datos (ORM)**

**`app/models/user_model.py`**
  Define los **modelos ORM** que representan las tablas de la base de datos.
  Por ejemplo, la tabla `users` con sus columnas y tipos de datos.

---


In [ ]:
from sqlalchemy import Column, Integer, String, DateTime, Boolean
from sqlalchemy.sql import func
from sqlalchemy.orm import validates
from core.database import Base
from datetime import datetime

class User(Base):
    """
    Modelo de usuario para la aplicación.
    
    Representa un usuario en el sistema con autenticación JWT.
    """
    __tablename__ = "users"
    
    # Campos principales
    id = Column(Integer, primary_key=True, index=True, comment="ID único del usuario")
    username = Column(
        String(50), 
        unique=True, 
        index=True, 
        nullable=False,
        comment="Nombre de usuario único"
    )
    hashed_password = Column(
        String(255), 
        nullable=False,
        comment="Contraseña hasheada del usuario"
    )
    
    # Campos de estado y roles
    role = Column(
        String(20), 
        default="user", 
        nullable=False,
        comment="Rol del usuario (user, admin)"
    )
    is_active = Column(
        Boolean, 
        default=True, 
        nullable=False,
        comment="Indica si el usuario está activo"
    )
    
    # Timestamps
    created_at = Column(
        DateTime(timezone=True), 
        server_default=func.now(),
        comment="Fecha y hora de creación del usuario"
    )
    updated_at = Column(
        DateTime(timezone=True), 
        onupdate=func.now(),
        comment="Fecha y hora de última actualización"
    )
    
    @validates('username')
    def validate_username(self, key, username):
        """Valida el formato del nombre de usuario"""
        if not username:
            raise ValueError("El nombre de usuario no puede estar vacío")
        if len(username) < 3:
            raise ValueError("El nombre de usuario debe tener al menos 3 caracteres")
        if len(username) > 50:
            raise ValueError("El nombre de usuario no puede tener más de 50 caracteres")
        if not username.replace('_', '').replace('-', '').isalnum():
            raise ValueError("El nombre de usuario solo puede contener letras, números, guiones y guiones bajos")
        return username.lower().strip()
    
    @validates('hashed_password')
    def validate_hashed_password(self, key, hashed_password):
        """Valida que la contraseña hasheada no esté vacía"""
        if not hashed_password:
            raise ValueError("La contraseña hasheada no puede estar vacía")
        return hashed_password
    
    def __repr__(self):
        """Representación string del objeto User"""
        return f"<User(id={self.id}, username='{self.username}')>"
    
    def to_dict(self):
        """Convierte el usuario a diccionario (sin contraseña)"""
        return {
            'id': self.id,
            'username': self.username,
            'role': self.role,
            'is_active': self.is_active,
            'created_at': self.created_at.isoformat() if self.created_at else None,
            'updated_at': self.updated_at.isoformat() if self.updated_at else None
        }


---

## **Paso 6. `schemas/` – Capa de validación y transferencia de datos (Pydantic)**

**`app/schemas/user_schemas.py`**
    Define los **esquemas Pydantic**, que se usan para validar la entrada y salida de datos en las rutas (no son tablas, sino estructuras de datos para la API).

---


In [ ]:
# app/schemas/user_schemas.py

from pydantic import BaseModel, Field, ConfigDict
from datetime import datetime
from typing import Optional

# ------------------------------------------------------------
# Schema para crear un nuevo usuario (registro)
class UserCreate(BaseModel):
    """Schema para el registro de nuevos usuarios"""
    username: str = Field(..., min_length=3, max_length=50, description="Nombre de usuario único", examples=["juan_perez"])
    password: str = Field(..., min_length=6, description="Contraseña del usuario", examples=["miPassword123"])
    role: str = Field(default="user", description="Rol del usuario", examples=["user", "admin"])

# ------------------------------------------------------------
# Schema para login de usuario
class UserLogin(BaseModel):
    """Schema para autenticación de usuarios"""
    username: str = Field(..., description="Nombre de usuario", examples=["juan_perez"])
    password: str = Field(..., description="Contraseña del usuario", examples=["miPassword123"])

# ------------------------------------------------------------
# Schema para la respuesta de usuario (sin contraseña)
class UserResponse(BaseModel):
    """Schema para la respuesta de datos de usuario"""
    id: int = Field(examples=[1])
    username: str = Field(examples=["juan_perez"])
    role: str = Field(examples=["user"])
    is_active: bool = Field(examples=[True])
    created_at: Optional[datetime] = Field(default=None, examples=["2025-10-09T14:30:00"])
    updated_at: Optional[datetime] = Field(default=None, examples=[None])
    
    model_config = ConfigDict(from_attributes=True)  # Permite crear desde modelos ORM

# ------------------------------------------------------------
# Schema para la respuesta del token de autenticación
class Token(BaseModel):
    """Schema para la respuesta de autenticación JWT"""
    access_token: str = Field(..., description="Token JWT de acceso", examples=["eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9..."])
    token_type: str = Field(default="bearer", description="Tipo de token", examples=["bearer"])

# ------------------------------------------------------------
# Schema para la respuesta completa de login (token + usuario)
class LoginResponse(BaseModel):
    """Schema para la respuesta de login con token y datos del usuario"""
    access_token: str = Field(..., description="Token JWT de acceso", examples=["eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9..."])
    token_type: str = Field(default="bearer", description="Tipo de token", examples=["bearer"])
    user: UserResponse = Field(..., description="Datos del usuario autenticado")

# ------------------------------------------------------------
# Schema para mensajes generales
class Message(BaseModel):
    """Schema para mensajes de respuesta"""
    mensaje: str = Field(examples=["Operación realizada exitosamente"])


---
## Paso 7: `routes/` – Capa de controladores o endpoints**

**`user_routes.py`**
  Define las **rutas HTTP (GET, POST, PUT, DELETE)** que los clientes pueden consumir.
  Aquí se importa la lógica de `auth`, `schemas`, `models` y `database`.

---


In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from sqlalchemy.orm import Session
from core.database import get_db
from models.user_model import User
from auth.auth_service import hashear_password, verificar_password
from auth.auth_handler import crear_token
from auth.dependencies import get_current_user
from schemas.user_schemas import UserCreate, UserLogin, UserResponse, Token, LoginResponse, Message

router = APIRouter(tags=["Autenticación"])  # Tag para documentación Swagger

@router.post("/login", response_model=LoginResponse, summary="Iniciar sesión")
def login(data: UserLogin, db: Session = Depends(get_db)):
    """
    Autentica un usuario y devuelve un token JWT junto con los datos del usuario.
    
    - **username**: Nombre de usuario
    - **password**: Contraseña del usuario
    
    Retorna:
    - **access_token**: Token JWT para autenticación
    - **token_type**: Tipo de token (bearer)
    - **user**: Datos del usuario autenticado
    """
    user = db.query(User).filter(User.username == data.username).first()
    
    if not user or not verificar_password(data.password, user.hashed_password):
        raise HTTPException(status_code=401, detail="Credenciales inválidas")

    token = crear_token({"sub": user.username})
    
    return LoginResponse(
        access_token=token,
        token_type="bearer",
        user=UserResponse.model_validate(user)
    )

@router.post("/register", response_model=UserResponse, status_code=201, summary="Registrar nuevo usuario")
def register(data: UserCreate, db: Session = Depends(get_db)):
    """
    Registra un nuevo usuario en el sistema.
    
    - **username**: Nombre de usuario único (mínimo 3 caracteres)
    - **password**: Contraseña (mínimo 6 caracteres)
    """
    user = db.query(User).filter(User.username == data.username).first()
    
    if user:
        raise HTTPException(status_code=400, detail="Usuario ya existe")

    hashed = hashear_password(data.password)

    nuevo_usuario = User(username=data.username, hashed_password=hashed, role=data.role)

    db.add(nuevo_usuario)
    db.commit()  # Confirmamos los cambios en la base de datos
    db.refresh(nuevo_usuario)  # Obtenemos la versión actualizada del usuario

    return UserResponse.model_validate(nuevo_usuario)

@router.get("/me", response_model=UserResponse, summary="Obtener usuario actual")
async def get_me(current_user: User = Depends(get_current_user)):
    """
    Obtiene los datos del usuario autenticado actualmente.
    
    Requiere autenticación mediante token JWT en el header:
    Authorization: Bearer <token>
    
    Retorna:
    - **id**: ID del usuario
    - **username**: Nombre de usuario
    - **is_active**: Estado del usuario
    - **created_at**: Fecha de creación
    - **updated_at**: Fecha de última actualización
    """
    return UserResponse.model_validate(current_user)

@router.get("/admin", summary="Ruta de administrador")
async def admin_route(current_user: User = Depends(get_current_user)):
    """
    Ruta protegida solo para administradores.
    
    Requiere autenticación mediante token JWT y rol de administrador.
    """
    if current_user.role != "admin":
        raise HTTPException(status_code=403, detail="Solo administradores pueden acceder a esta ruta")
    
    return {"message": f"Bienvenido, administrador {current_user.username}"}


---

### Paso 8: `app/main.py`




In [ ]:
# app/main.py

# Importamos FastAPI para crear la aplicación web y HTTPException para manejo de errores
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse

# Importamos middleware de CORS para permitir peticiones desde otros dominios
from fastapi.middleware.cors import CORSMiddleware

# Importamos la base declarativa y el motor de la base de datos
from core.database import Base, engine

# Importamos el modelo User para que se cree la tabla
from models.user_model import User

# Importamos las rutas de usuario con un alias
from routes.user_routes import router as user_router

# Importamos logging para registrar mensajes en consola o archivos
import logging

# Importamos contextlib para manejar eventos de ciclo de vida
from contextlib import asynccontextmanager

# ------------------------------------------------------------
# Configuración del sistema de logging para monitoreo de errores
logging.basicConfig(
    level=logging.INFO,  # Nivel de registro (INFO, DEBUG, ERROR, etc.)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'  # Formato del mensaje
)
logger = logging.getLogger(__name__)  # Creamos un logger específico para este archivo

# ------------------------------------------------------------
# Gestor de eventos de ciclo de vida de la aplicación
@asynccontextmanager
async def lifespan(app: FastAPI):
    """Gestiona el inicio y cierre de la aplicación"""
    # Código que se ejecuta al iniciar
    try:
        Base.metadata.create_all(bind=engine)  # Ejecuta la creación de todas las tablas definidas en los modelos
        logger.info("Base de datos inicializada correctamente")  # Mensaje de éxito
    except Exception as e:
        logger.error(f"Error al inicializar la base de datos: {str(e)}")  # Mensaje de error
        raise  # Lanza nuevamente la excepción para detener la aplicación si hay fallo
    
    yield  # Aquí la aplicación está en ejecución
    
    # Código que se ejecuta al cerrar (si fuera necesario en el futuro)
    logger.info("Aplicación finalizando...")

# ------------------------------------------------------------
# Instancia principal de la aplicación FastAPI
app = FastAPI(
    title="Sistema de Gestión de Cuentas",  # Título que aparecerá en la documentación Swagger
    description="API para gestión de usuarios con autenticación JWT",  # Descripción de la API
    version="1.0.0",  # Versión de la API
    lifespan=lifespan  # Gestor de ciclo de vida
)

# ------------------------------------------------------------
# Middleware para permitir CORS (Cross-Origin Resource Sharing)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Permite peticiones desde cualquier origen (¡modificar en producción!)
    allow_credentials=True,  # Permite el envío de cookies o credenciales
    allow_methods=["*"],  # Permite todos los métodos HTTP (GET, POST, etc.)
    allow_headers=["*"],  # Permite todos los encabezados personalizados
)

# ------------------------------------------------------------
# Manejador global de errores no controlados
@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    logger.error(f"Error no manejado: {str(exc)}")  # Registra el error
    return JSONResponse(
        status_code=500,  # Error interno del servidor
        content={"detail": "Error interno del servidor"}
    )

# ------------------------------------------------------------
# Incluir el conjunto de rutas definidas en el archivo user_routes
app.include_router(user_router, prefix="/api/v1")  # Todas las rutas estarán bajo /api/v1 (ej: /api/v1/login)

# ------------------------------------------------------------
# Punto de entrada principal de la aplicación
def main():
    """Función principal para iniciar el servidor"""
    import uvicorn
    import webbrowser
    from threading import Timer
    
    host = "127.0.0.1"
    port = 8000
    
    # Abrir navegador automáticamente después de 1.5 segundos
    def open_browser():
        webbrowser.open(f"http://{host}:{port}/docs")
    
    Timer(1.5, open_browser).start()
    
    # Ejecutar servidor uvicorn con hot-reload
    uvicorn.run(
        "main:app",  # Módulo:aplicación
        host=host,  # Escucha en localhost
        port=port,  # Puerto del servidor
        reload=True,  # Hot-reload activado para desarrollo
        log_level="info"  # Nivel de logs
    )

if __name__ == "__main__":
    main()


---

### Paso 9: `app/streamlit_app.py`




In [ ]:
import streamlit as st
import requests

API_BASE_URL = "http://localhost:8000/api/v1"

st.set_page_config(page_title="Gestión de Cuentas", page_icon="🔐")
st.title("Sistema de Gestión de Cuentas")

menu = ["Registro", "Login"]
choice = st.sidebar.selectbox("Menú", menu)

# Inicializar variables de sesión
if "token" not in st.session_state:
    st.session_state.token = None
if "user" not in st.session_state:
    st.session_state.user = None

if choice == "Registro":
    st.subheader("Crear nuevo usuario")
    
    # Formulario de registro
    with st.form("register_form", clear_on_submit=True):
        username = st.text_input("Usuario", help="3-50 caracteres")
        password = st.text_input("Contraseña", type="password", help="Mínimo 6 caracteres")
        role = st.selectbox("Rol", ["user", "admin"], help="Selecciona el rol del usuario")
        submit = st.form_submit_button("Registrar")
        
        if submit:
            if len(username) < 3 or len(username) > 50:
                st.error("El usuario debe tener entre 3 y 50 caracteres")
            elif len(password) < 6:
                st.error("La contraseña debe tener al menos 6 caracteres")
            else:
                data = {"username": username, "password": password, "role": role}
                try:
                    r = requests.post(f"{API_BASE_URL}/register", json=data)
                    if r.status_code == 201:
                        st.success(f"✅ Usuario '{username}' creado correctamente")
                    else:
                        try:
                            error_detail = r.json().get('detail', r.text)
                            st.error(f"❌ Error: {error_detail}")
                        except:
                            st.error(f"❌ Error: {r.text}")
                except Exception as e:
                    st.error(f"❌ Error de conexión: {e}")

if choice == "Login":
    st.subheader("Iniciar sesión")
    
    # Formulario de login
    with st.form("login_form", clear_on_submit=True):
        username = st.text_input("Usuario")
        password = st.text_input("Contraseña", type="password")
        submit = st.form_submit_button("Entrar")
        
        if submit:
            if not username or not password:
                st.warning("⚠️ Completa usuario y contraseña")
            else:
                data = {"username": username, "password": password}
                try:
                    r = requests.post(f"{API_BASE_URL}/login", json=data)
                    if r.status_code == 200:
                        token = r.json()["access_token"]
                        st.session_state.token = token
                        st.success("✅ Login exitoso")
                        
                        # Obtener datos de usuario
                        headers = {"Authorization": f"Bearer {token}"}
                        r2 = requests.get(f"{API_BASE_URL}/me", headers=headers)
                        if r2.status_code == 200:
                            st.session_state.user = r2.json()
                        else:
                            st.session_state.user = None
                    else:
                        st.error("❌ Credenciales inválidas")
                except Exception as e:
                    st.error(f"❌ Error de conexión: {e}")

    # Mostrar información del usuario logueado
    if st.session_state.token and st.session_state.user:
        # Mostrar información específica según el rol
        if st.session_state.user['role'] == "admin":
            # Probar acceso a ruta de administrador
            headers = {"Authorization": f"Bearer {st.session_state.token}"}
            try:
                r = requests.get(f"{API_BASE_URL}/admin", headers=headers)
                if r.status_code == 200:
                    st.success("✅ Acceso confirmado a funciones de administrador")
                else:
                    st.warning("⚠️ No tienes acceso a funciones de administrador")
            except Exception as e:
                st.error(f"❌ Error al verificar acceso: {e}")
        
        # Botón para cerrar sesión
        if st.button("🚪 Cerrar Sesión"):
            st.session_state.token = None
            st.session_state.user = None
            st.success("✅ Sesión cerrada correctamente")
            st.rerun()


---

### Ejecutar el proyecto


1. Asegúrate de tener PostgreSQL encendido y crea la base de datos:

```bash
      autenticacion
```

2. Ejecuta el BackEnd:

```bash
      python main.py
```

3. Ejecuta el FrontEnd:

```bash
      streamlit run streamlit_app.py
```
---


